In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase

tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")


@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [3]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [4]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role
    
    if user_role == "internal":
        pass # internal users get access to all tools
    else:
        tools = [web_search] # external users only get access to web search
        request = request.override(tools=tools) 

    return handler(request)

In [5]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [6]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "external"}
)

print(response["messages"][-1].content)

I don’t have access to your database, so I can’t give the exact count. If you tell me what system you’re using (SQL database like MySQL/PostgreSQL/SQLite, MongoDB, etc.) and the table/collection name, I can provide the exact query. Here are common options you can run:

- SQL (table named artists)
  - Total artists (including duplicates if duplicates exist in rows):
    SELECT COUNT(*) AS artist_count FROM artists;
  - Distinct artists (based on an id column, e.g., artist_id):
    SELECT COUNT(DISTINCT artist_id) AS artist_count FROM artists;
  - Active artists only (if you have a status/active flag):
    SELECT COUNT(*) AS artist_count FROM artists WHERE is_active = TRUE;

- MongoDB (collection named artists)
  - Total documents (artists):
    db.artists.countDocuments({});
  - Distinct IDs (if you want unique artists by _id):
    db.artists.distinct("_id").length

- Python/pandas (assuming you’ve loaded data into a DataFrame)
  - Total rows:
    len(artists_df)
  - Distinct by a key (

In [7]:
response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "internal"}
)

print(response["messages"][-1].content)

275 artists.

Note: The proper table is Artist (not artists). Running SELECT COUNT(*) FROM Artist returns 275. If you’d like, I can fetch a list of artist names or other statistics.
